In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import tensorflow.keras as keras

from matplotlib import pyplot as plt

DATASET = 'tsp448/'
TFRD_ROOT = '/kaggle/input/'+DATASET  #dataset path
MODEL_ROOT = '/kaggle/input/models/'  #saved model path
AUTOTUNE = tf.data.AUTOTUNE           # -1

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver().connect()
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    tpu = None
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

if tpu:
    from kaggle_secrets import UserSecretsClient
    from kaggle_datasets import KaggleDatasets
    user_secrets = UserSecretsClient()
    user_credential = user_secrets.get_gcloud_credential()
    user_secrets.set_tensorflow_credential(user_credential)
    TFRD_ROOT = KaggleDatasets().get_gcs_path(DATASET[:-1])

In [ ]:
class TFRecordReader: 
    def __init__(self,filepath):
        self.filepath = filepath

    def decode_map(self):
        feature_description = {
            'img':tf.io.FixedLenFeature([], tf.string),
            'label':tf.io.FixedLenFeature([], tf.float32)
            }
        def _decode_map(raw):
            d = tf.io.parse_single_example(raw,feature_description)
            img = tf.io.parse_tensor(d['img'],tf.float32)
            img = tf.reshape(img,INPUT_SIZE)
            return img,d['label']
        return _decode_map
    
    def get_pipeline(self):
        return tf.data.TFRecordDataset(self.filepath,'ZLIB',num_parallel_reads=AUTOTUNE
                                                    ).map(self.decode_map(),AUTOTUNE)
    
def augment_img_tensor_map(img,label):
    return tf.image.rot90(img, np.random.choice([0,1,2,3])),label

In [ ]:
def show_history(history):
    plt.ylim([0,1000])
    plt.plot(history.history['RMSE'])
    plt.plot(history.history['val_RMSE'])
    plt.show()

In [ ]:
TARGET_SIZE = (448,448)
INPUT_SIZE = (448,448,3)
MODEL_NAME = 'res50_5_23'

if tpu:
    BATCH_SIZE = 128 * strategy.num_replicas_in_sync
else:
    BATCH_SIZE = 32

**Input pipelines**

If tpu is used, there will be a lot of RAM, so cache all the data first.

In [ ]:
pipeline = TFRecordReader(TFRD_ROOT+'/tsp_train.tfrecord').get_pipeline()
val_pipeline = TFRecordReader(TFRD_ROOT+'/tsp_val.tfrecord').get_pipeline()
if tpu:
    pipeline = pipeline.cache(
                            ).shuffle(4096, reshuffle_each_iteration=True
                            ).map(augment_img_tensor_map,num_parallel_calls=AUTOTUNE,deterministic=False
                            ).batch(BATCH_SIZE,drop_remainder=True
                            ).prefetch(AUTOTUNE)
    val_pipeline = val_pipeline.batch(82,drop_remainder=True
                                ).cache(
                                ).prefetch(AUTOTUNE)
else:
    pipeline = pipeline.shuffle(256
                            ).map(augment_img_tensor_map,num_parallel_calls=AUTOTUNE
                            ).batch(BATCH_SIZE,drop_remainder=True
                            ).prefetch(AUTOTUNE)
    val_pipeline = val_pipeline.batch(BATCH_SIZE
                                ).cache(
                                ).prefetch(AUTOTUNE)

**Callbacks**

check_callback for save best model 

reduce_callback for reducing the learning rate when the loss did not drop

In [ ]:
check_callback = keras.callbacks.ModelCheckpoint(MODEL_NAME+'.h5',save_best_only=True) 
reduce_callback = keras.callbacks.ReduceLROnPlateau('loss',factor=0.5,min_lr=1e-10,patience=6,verbose=1)
callbacks = [check_callback]

**Model**

Since the features in the image is simple, we only use the 1,2,3 stage of res50 network for feature extraction. 

It's much faster than the full res50 network, and did not loss many accuracy.

Use Cov2D layer with 5x5 kernel size in first stage to get more details about the image.

In [ ]:
def res50_5_23():
    def identity_block(input_tensor, kernel_size, filters):
        filters1, filters2, filters3 = filters

        x = keras.layers.Conv2D(filters1, (1, 1),kernel_initializer='he_normal')(input_tensor)
        x = keras.layers.BatchNormalization(axis=3)(x)
        x = keras.layers.Activation('relu')(x)

        x = keras.layers.Conv2D(filters2, kernel_size, padding='same', kernel_initializer='he_normal')(x)
        x = keras.layers.BatchNormalization(axis=3)(x)
        x = keras.layers.Activation('relu')(x)

        x = keras.layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal')(x)
        x = keras.layers.BatchNormalization(axis=3)(x)

        x = keras.layers.add([x, input_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def conv_block(input_tensor,
                kernel_size,
                filters,
                strides=(2, 2),padding='same'):
        filters1, filters2, filters3 = filters

        x = keras.layers.Conv2D(filters1, (1, 1), strides=strides,
                        kernel_initializer='he_normal')(input_tensor)
        x = keras.layers.BatchNormalization(axis=3)(x)
        x = keras.layers.Activation('relu')(x)

        x = keras.layers.Conv2D(filters2, kernel_size, padding=padding, kernel_initializer='he_normal')(x)
        x = keras.layers.BatchNormalization(axis=3)(x)
        x = keras.layers.Activation('relu')(x)

        x = keras.layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal')(x)
        x = keras.layers.BatchNormalization(axis=3)(x)

        shortcut = keras.layers.Conv2D(filters3, (1, 1), strides=strides,kernel_initializer='he_normal')(input_tensor)
        shortcut = keras.layers.BatchNormalization(axis=3)(shortcut)

        x = keras.layers.add([x, shortcut])
        x = keras.layers.Activation('relu')(x)
        return x

    input_layer = keras.layers.Input(INPUT_SIZE)
    x = keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal")(input_layer)
    
    x = keras.layers.ZeroPadding2D(2)(x)
    x = keras.layers.Conv2D(64,5,2,kernel_initializer='he_normal')(x)
    x = keras.layers.BatchNormalization(axis=3)(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D()(x)

    x = conv_block(x, 3, [64, 64, 256], strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256])
    x = identity_block(x, 3, [64, 64, 256])

    x = conv_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])

    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(512,activation=keras.activations.relu)(x)
    x = keras.layers.Dropout(0.2)(x)
    output = keras.layers.Dense(1)(x)

    return keras.Model(inputs=input_layer,outputs=output)

In [ ]:
with strategy.scope():
    model = res50_5_23()
    model.compile(keras.optimizers.Adam(),loss=keras.losses.mse,metrics=[keras.metrics.RootMeanSquaredError(name='RMSE')])
model.build(INPUT_SIZE) 
model.summary()

In [ ]:
history = model.fit(pipeline,epochs=150,validation_data=val_pipeline,callbacks=callbacks)#validation_data=val_pipeline
show_history(history)

**Validation and Prediction**

Following parts cannot run with TPU

In [ ]:
def val_result(model,df:pd.DataFrame):
    y_pred = calc_result(model,df)
    y_true = df['distance'].to_numpy(np.float32)
    rmse = keras.losses.mse(y_true,y_pred)**(1/2)
    print(rmse)

def calc_result(model:keras.Model,df:pd.DataFrame):
    arr_scale = []
    def _gen():
        for file in df['filename']:
            img = load_jpeg(INPUT_ROOT+file)
            img,scale = resize_img_tensor(img,TARGET_SIZE)
            img = img/127.5 - 1
            arr_scale.append(scale)
            #print('\r',file,end='')
            yield img
    
    x = tf.data.Dataset.from_generator(_gen,tf.float32).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    y_pred = model.predict(x)
    y_pred = np.asarray(y_pred,np.float32)
    y_pred = y_pred.reshape((-1))
    scale = np.asarray(arr_scale,np.float32)
    y_pred = y_pred * scale
    return y_pred


INPUT_ROOT = '/kaggle/input/tsp-cv/' #img path
train_csv_path = INPUT_ROOT+'train.csv'
test_csv_path = INPUT_ROOT+'test.csv'
df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

if os.path.exists(MODEL_NAME+'.h5'):
    model = keras.models.load_model(MODEL_NAME+'.h5')

val_result(model,df[:512])  #training set
val_result(model,df[-512:]) #validation set

In [ ]:
pred_y = calc_result(model,test_df)
test_df['distance'] = pred_y
test_df = test_df[['id','distance']]
test_df.to_csv('submit.csv')